In [1]:
from datetime import timedelta, datetime
import os
import glob

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

import random
import pickle
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

# 일수 계산
import calendar

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity

SEED = 42

def seed_all(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

seed_all(SEED)

### 데이터 로드 및 전처리

In [2]:
seed_all(SEED)

ori_new_df = pd.read_csv('./New_Data/MES_data_argumentation_126233ea_from_1193ea.csv')
tmp_data_df = ori_new_df.drop(columns=['호기','Date', 'type', 'ID'])
oriCol_ls = tmp_data_df.columns.tolist()

# 시간 정보 -> timestamp화
tmp_data_df['출탕시간'] = pd.to_datetime(tmp_data_df['출탕시간']).values.astype(np.int64) // 10**9
tmp_data_df['홀딩로투입시간'] = pd.to_datetime(tmp_data_df['홀딩로투입시간']).values.astype(np.int64) // 10**9

tmp_data_df

,Charge,출탕량,출탕시간,홀딩로투입시간,출탕투입시간차,실질홀딩로투입온도,Metal Temp Set,Metal Temp,연주냉각 前 냉각수온도,연주냉각 後 냉각수온도,WITHDROW,Waiting Time,w.kg,mm/min,V%W,V%B,year,month,day,출탕_hour,출탕_minute,출탕_sec,투입_hour,투입_minute,투입_sec
0,24,1101.0,1660236960,1660239120,6.0,1343.0,829.0,826.0,42.0,63.0,60.0,6.0,28154.0,540.0,60.0,60.0,2023,8.0,11.0,18.0,2.0,0.0,19.0,20.0,0.0
1,23,1162.0,1663605960,1663609080,10.0,1333.0,810.0,807.0,55.0,65.0,60.0,5.5,26270.0,660.0,60.0,60.0,2023,9.0,19.0,17.0,10.0,0.0,16.0,38.0,0.0
2,29,1108.0,1687807800,1687808400,10.0,1320.0,1009.0,1009.0,38.0,64.0,60.0,4.8,149.0,600.0,60.0,60.0,2022,6.0,26.0,19.0,29.0,0.0,19.0,40.0,0.0
3,4,1117.0,1669891260,1669894140,7.0,1326.0,972.0,995.0,23.5,67.0,60.0,4.6,3769.0,600.0,60.0,60.0,2023,12.0,1.0,10.0,24.0,0.0,11.0,29.0,0.0
4,10,1104.0,1687181580,1687181940,12.0,1283.0,980.0,979.0,44.0,65.0,60.0,5.5,2378.0,540.0,60.0,60.0,2022,6.0,19.0,14.0,26.0,0.0,18.0,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126228,24,1105.0,1675792800,1675794780,14.0,1328.0,983.0,980.0,42.0,65.0,60.0,5.0,26316.0,600.0,60.0,60.0,2022,2.0,7.0,18.0,38.0,0.0,18.0,27.0,0.0
126229,7,1141.0,1684147080,1684148160,10.0,1338.0,1079.0,1070.0,32.0,66.0,60.0,5.5,3568.0,480.0,60.0,60.0,2023,5.0,15.0,11.0,20.0,0.0,11.0,9.0,0.0
126230,27,1114.0,1663611000,1663611720,12.0,1333.0,814.0,813.0,55.0,64.0,60.0,6.0,30824.0,420.0,60.0,60.0,2023,9.0,19.0,20.0,8.0,0.0,19.0,6.0,0.0
126231,17,1112.0,1665587940,1665590880,9.0,1325.0,1060.0,1064.0,42.0,65.0,60.0,4.8,22151.0,600.0,60.0,60.0,2022,10.0,12.0,12.0,59.0,0.0,16.0,14.0,0.0


### 아웃라이어 생성

In [3]:
# def generate_outlier_from_normal_value(mean, std, outlier_size, lower_bound, upper_bound, out_err_low=0.2, out_err_high=0.5):

#     fin_outliers = np.empty()
#     while len(fin_outliers) < outlier_size:
#         outliers = np.random.normal(loc=mean, scale=std*2, size=outlier_size*100)

#         # 원본 데이터 범위 기반 이상치 재분류
#         lower_bound = lower_bound - lower_bound*out_err_low
#         upper_bound = upper_bound - upper_bound*out_err_high
#         lower_outliers = outliers[outliers < lower_bound]
#         upper_outliers = outliers[outliers > upper_bound]

#         fin_outliers = np.concatenate((fin_outliers, lower_outliers, upper_outliers))
#         np.random.shuffle(fin_outliers)

#     return outliers

# def out_generate(mean, outlier_err, outlier_size=10, out_err_low=1.0, out_err_high=1.5):
#     # 에러 기본 크기 (평균과 최대값간 거리)
#     lower_errs = outlier_err * np.random.uniform(low=0.7, high=1.0, size=outlier_size)
#     lower_outliers = mean - lower_errs

#     upper_errs = outlier_err * np.random.uniform(low=out_err_low, high=out_err_high, size=outlier_size)
#     upper_outliers = mean + upper_errs

#     data = np.concatenate((lower_outliers, upper_outliers))
#     np.random.shuffle(data)

#     return data

def generate_outlier_from_normal_dist(mean, std, outlier_size, out_err_low=0.7, out_err_high=1.0):
    outlier_errs = np.random.uniform(low=0.7, high=1.0, size=outlier_size)
    lower_errs = outlier_errs * np.random.normal(loc=mean, scale=std, size=outlier_size)
    lower_outliers = mean - lower_errs
    
    outlier_errs = np.random.uniform(low=out_err_low, high=out_err_high, size=outlier_size)
    upper_errs = outlier_errs * np.random.normal(loc=mean, scale=std, size=outlier_size)
    upper_outliers = mean + upper_errs

    data = np.concatenate((lower_outliers, upper_outliers))
    np.random.shuffle(data)

    return data

In [4]:
seed_all(SEED)

# 20퍼센트 아웃라이어 생성
outlier_num = int(len(tmp_data_df) * 0.2)
new_out_df = pd.DataFrame()

time_cols = []

i = 1
while len(new_out_df) <= outlier_num:
    # print(f'Num of outlier: {len(new_out_df)} at {i}th')
    tmp_out_df = pd.DataFrame()
    
    # 이상치 생성
    for col in tmp_data_df.columns:
        mean = tmp_data_df[col].mean()
        std = tmp_data_df[col].std()
        tmp_out_df[col] = generate_outlier_from_normal_dist(mean, std, outlier_num, 1.0, 1.5)

    # 불가능한 음수 값 제거
    no_neg_cols = ['Charge','출탕량','출탕시간','홀딩로투입시간','출탕투입시간차','WITHDROW','Waiting Time','w.kg','mm/min','V%W','V%B']
    for col in no_neg_cols:
        tmp_out_df = tmp_out_df[tmp_out_df[col] >= 0]
        # 마이너스 부호 제거
        tmp_out_df[col] = tmp_out_df[col].abs()

    # 출탕 투입시간 비교
    tmp_out_df = tmp_out_df[tmp_out_df['출탕시간'] <= tmp_out_df['홀딩로투입시간']]

    # 날짜 범위 체크 + 정수형으로 변경 및 마이너스 부호 제거
    tmp_out_df['year'] = np.random.choice(tmp_data_df['year'].unique().astype('int').tolist(), len(tmp_out_df))
    tmp_out_df['month'] = tmp_out_df['month'].round().abs()
    tmp_out_df['day'] = tmp_out_df['day'].round().abs()
    tmp_out_df = tmp_out_df[(tmp_out_df['month'] >= 1)&(tmp_out_df['month'] <= 12)]

    # 시간 범위 체크 + 정수형으로 변경 및 마이너스 부호 제거
    for time_col in ['출탕_hour', '출탕_minute', '출탕_sec', '투입_hour', '투입_minute', '투입_sec']:
        tmp_out_df[time_col] = tmp_out_df[time_col].round().abs()

    tmp_out_df = tmp_out_df[(tmp_out_df['출탕_hour'] >= 0)&(tmp_out_df['출탕_hour'] <= 23)]
    tmp_out_df = tmp_out_df[(tmp_out_df['출탕_minute'] >= 0)&(tmp_out_df['출탕_minute'] < 60)]
    tmp_out_df = tmp_out_df[(tmp_out_df['출탕_sec'] >= 0)&(tmp_out_df['출탕_sec'] < 60)]

    tmp_out_df = tmp_out_df[(tmp_out_df['투입_hour'] >= 0)&(tmp_out_df['투입_hour'] <= 23)]
    tmp_out_df = tmp_out_df[(tmp_out_df['투입_minute'] >= 0)&(tmp_out_df['투입_minute'] < 60)]
    tmp_out_df = tmp_out_df[(tmp_out_df['투입_sec'] >= 0)&(tmp_out_df['투입_sec'] < 60)]

    tmp_out_df['max_day'] = tmp_out_df.apply(lambda x: calendar.monthrange(int(x['year']), int(x['month']))[1], axis=1)
    # tmp_out_df['max_day'] = tmp_data_df.apply(lambda x: calendar.monthrange(int(x['year']), int(x['month']))[1], axis=1).tolist()
    tmp_out_df = tmp_out_df[tmp_out_df['day'] <= tmp_out_df['max_day']]

    new_out_df = pd.concat([new_out_df, tmp_out_df[oriCol_ls]], ignore_index=True)
    new_out_df.drop_duplicates(ignore_index=True, inplace=True)

    i+=1

print(f'Num of fin outlier: {len(new_out_df)} ({len(new_out_df)/len(tmp_data_df)*100:0.2f}% of Normal data: {len(tmp_data_df)})')
new_out_df

Num of fin outlier: 25917 (20.53% of Normal data: 126233)


,Charge,출탕량,출탕시간,홀딩로투입시간,출탕투입시간차,실질홀딩로투입온도,Metal Temp Set,Metal Temp,연주냉각 前 냉각수온도,연주냉각 後 냉각수온도,WITHDROW,Waiting Time,w.kg,mm/min,V%W,V%B,year,month,day,출탕_hour,출탕_minute,출탕_sec,투입_hour,투입_minute,투입_sec
0,9.108680,273.140203,1.155136e+08,3.962883e+09,24.855144,191.342874,139.479106,276.944568,100.924461,156.334898,1.452392,13.066158,22699.755462,1413.003221,15.906366,139.865943,2023,1.0,27.0,0.0,11.0,0.0,1.0,1.0,0.0
1,6.141352,2479.896185,4.312945e+08,4.894377e+08,22.774675,2890.464954,2079.412630,58.912710,82.311930,130.692613,143.976098,11.494848,7347.428854,1183.162841,144.457036,11.467902,2022,11.0,5.0,6.0,22.0,0.0,3.0,30.0,0.0
2,5.535394,274.058359,3.896763e+08,4.108518e+09,2.703797,3224.677657,2234.729239,192.879605,95.611457,7.527632,140.846276,10.022702,1722.985729,1194.678467,149.253570,8.962268,2023,1.0,9.0,5.0,6.0,0.0,1.0,0.0,0.0
3,35.779290,318.039359,3.215797e+08,3.989606e+09,0.706205,345.384316,2130.765091,2107.576826,14.003265,6.557191,15.317758,1.490822,32021.719392,149.695372,146.438224,126.535722,2022,2.0,3.0,4.0,7.0,0.0,6.0,5.0,0.0
4,2.451170,2776.892877,3.507643e+09,3.592219e+09,20.736873,125.476368,2171.894337,57.275304,80.350237,144.914319,2.037334,11.552502,24191.012440,1391.689360,127.226135,15.654838,2022,11.0,10.0,10.0,8.0,0.0,0.0,29.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25912,9.189689,155.878947,2.255328e+07,1.107018e+08,1.226849,208.395859,2167.362636,98.195078,111.739112,144.782055,16.816361,1.086054,23914.896320,1245.012399,120.829394,15.827322,2022,9.0,3.0,5.0,35.0,0.0,2.0,20.0,0.0
25913,33.882475,2402.937293,5.884535e+07,1.556184e+08,20.819610,136.795072,1858.258845,217.218493,108.694178,141.420824,16.774811,10.119156,14888.184894,190.748404,129.891821,136.082594,2023,1.0,4.0,7.0,4.0,0.0,1.0,11.0,0.0
25914,5.818449,211.075629,1.095968e+08,3.937109e+09,16.907485,190.567145,134.148743,209.577227,75.587595,155.179971,132.925807,11.332019,36557.825385,2.126469,128.846404,8.722649,2022,6.0,3.0,2.0,13.0,0.0,3.0,33.0,0.0
25915,11.773680,186.847708,2.036214e+08,3.705972e+09,21.225956,233.350915,304.802096,2225.542110,83.471123,18.296039,10.892210,0.051450,18480.723006,1272.594735,15.549097,7.155751,2023,10.0,8.0,0.0,48.0,0.0,2.0,13.0,0.0


### 후처리

In [5]:
seed_all(SEED)
new_out_df['type'] = 'Outlier'

# Charge to be integer
new_out_df['Charge'] = new_out_df['Charge'].round()

# convert seconds into datetime
new_out_df['출탕시간'] = new_out_df['출탕시간'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')).apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
new_out_df['홀딩로투입시간'] = new_out_df['홀딩로투입시간'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')).apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))

# 차지 정수형 변경 및 문자열로 표현
new_out_df['Charge'] = new_out_df['Charge'].apply(lambda x: f'{int(x):02d}')
# 날짜 복원
new_out_df['Date'] = new_out_df['출탕시간'].apply(lambda x: x.strftime('%Y-%m-%d'))
# 호기 추가
new_out_df['호기'] = 2
# ID 추가
new_out_df['ID'] = new_out_df['Date'].str.replace('-','') + '_' + new_out_df['Charge']

new_data_df = pd.concat([ori_new_df, new_out_df], ignore_index=True)

ordered_cols = ['type', 'ID', 'Date', '호기', 
                'Charge', '출탕량', '출탕시간', '홀딩로투입시간', '출탕투입시간차', 
                '실질홀딩로투입온도', 'Metal Temp Set', 'Metal Temp', 
                '연주냉각 前 냉각수온도', '연주냉각 後 냉각수온도', 'WITHDROW', 'Waiting Time', 'w.kg', 'mm/min', 'V%W', 'V%B',
                'year', 'month', 'day', '출탕_hour', '출탕_minute', '출탕_sec', '투입_hour', '투입_minute', '투입_sec'
                ]

new_data_df = new_data_df[ordered_cols]
# data shuffling하고 index reset
new_data_df = new_data_df.sample(frac=1).reset_index(drop=True)

# 셔플
new_data_df.to_csv(f'./New_Data/MES_data_argumentation_with_outliers_{len(new_data_df)}ea.csv', index=False, encoding="utf-8-sig")
print(f'원본 및 증강, 이상치 데이터 결과: {new_data_df.shape}({len(new_out_df)/len(ori_new_df)*100:.2f})%')
display(new_data_df.head())

원본 및 증강, 이상치 데이터 결과: (152150, 29)(20.53)%


,type,ID,Date,호기,Charge,출탕량,출탕시간,홀딩로투입시간,출탕투입시간차,실질홀딩로투입온도,Metal Temp Set,Metal Temp,연주냉각 前 냉각수온도,연주냉각 後 냉각수온도,WITHDROW,Waiting Time,w.kg,mm/min,V%W,V%B,year,month,day,출탕_hour,출탕_minute,출탕_sec,투입_hour,투입_minute,투입_sec
0,Normal,20221012_20,2022-10-12,2,20,1108.000000,2022-10-12 16:44:00,2022-10-12 16:52:00,8.000000,1326.000000,1060.000000,1063.000000,42.000000,65.000000,60.000000,4.800000,23306.000000,600.000000,60.000000,60.000000,2023,10.0,12.0,15.0,44.0,0.0,15.0,52.0,0.0
1,Outlier,19820729_24,1982-07-29,2,24,2276.222344,1982-07-29 09:10:17,2098-09-11 11:18:40,4.545021,2975.715746,1948.712308,184.160388,79.405555,159.431455,132.133883,11.557039,13272.927003,1198.543609,142.370267,148.064285,2023,1.0,23.0,2.0,5.0,0.0,5.0,43.0,0.0
2,Normal,20221024_10,2022-10-24,2,10,1145.000000,2022-10-24 11:01:00,2022-10-24 11:10:00,9.000000,1326.000000,1116.000000,1115.000000,26.000000,65.000000,60.000000,5.000000,6041.000000,600.000000,60.000000,60.000000,2023,10.0,24.0,11.0,1.0,0.0,11.0,10.0,0.0
3,Normal,20221101_14,2022-11-01,2,14,1105.000000,2022-11-01 16:35:00,2022-11-01 16:44:00,9.000000,1325.000000,900.000000,896.000000,30.000000,68.000000,60.000000,5.000000,24013.000000,600.000000,60.000000,60.000000,2022,11.0,1.0,14.0,35.0,0.0,16.0,44.0,0.0
4,Outlier,19750512_20,1975-05-12,2,20,2232.637527,1975-05-12 03:43:40,2089-02-12 00:07:16,29.505547,2769.742995,168.969384,2447.387392,-0.235222,12.410609,146.894370,2.219396,22403.407280,110.231840,133.104343,2.333143,2023,5.0,28.0,2.0,7.0,0.0,6.0,23.0,0.0


In [6]:
print(f'총 데이터 수: {len(new_data_df)}')
print(f'원본 데이터 수: {len(new_data_df[new_data_df["type"]=="Original"])}')
print(f'생성 정상 데이터 수: {len(new_data_df[new_data_df["type"]=="Normal"])}')
print(f'생성 이상 데이터 수: {len(new_data_df[new_data_df["type"]=="Outlier"])}')

총 데이터 수: 152150
원본 데이터 수: 1193
생성 정상 데이터 수: 125040
생성 이상 데이터 수: 25917
